# GenerativeAI "Sprachmodell" Projekt

## Verbing mit wandb

In [1]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: adel-haj-jumah (adel-haj-jumah-hochschule-hannover) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

1. Importieren die notwendigen Bibliotheken

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
import random
from datasets import load_dataset
import torch.optim as optim


2. Token And Positional Embedding

In [10]:

class TokenAndPositionalEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, max_len):
        super().__init__()
        # ID der Token in einen Vektorraum
        self.token_embed = nn.Embedding(vocab_size, d_model)
        # Positionale Einbettungen (lernen relative Positionen "das ist learnable")
        self.pos_embedding = nn.Parameter(torch.randn(1, 512, d_model)) # batch 1 , bis zu 512 token und vektor größe
    def forward(self, x):
        seq_len = x.size(1) # 1 ist Anzahl der Token"Sequenzlänge" (0 ist batch)
        token_emb = self.token_embed(x)
        pos_emb = self.pos_embedding[:, :seq_len, :]
        return token_emb + pos_emb

3. Masked Multi-Head Self-Attention


In [4]:

class MaskedSelfAttention(nn.Module):
    """
    Masked (Causal) Self-Attention Layer
    → Modell kann nur auf vergangene Tokens schauen
    → Verwendet PyTorch nn.MultiheadAttention
    """
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.attn = nn.MultiheadAttention(
            embed_dim=d_model, 
            num_heads=n_heads, 
            batch_first=True  # wichtig! Damit x.shape = (B, T, C) funktioniert
        )

    def forward(self, x):
        T = x.size(1)  # Sequence Length

        # Causal Mask (obere Dreiecksmatrix)
        mask = torch.triu(torch.ones(T, T), diagonal=1).bool().to(x.device)

        # MultiheadAttention erwartet: (query, key, value, attn_mask)
        out, _ = self.attn(x, x, x, attn_mask=mask)
        return out


4. Add & Norm

In [5]:
class AddNorm(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, sublayer_output):
        return self.norm(x + sublayer_output)

5. Feedforward Layer

In [6]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

    def forward(self, x):
        return self.net(x)

6. Alles zusammenfügen: Transformator-Decoderblock

In [7]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.attn = MaskedSelfAttention(d_model, n_heads)  # PyTorch MultiheadAttention inside
        self.addnorm1 = AddNorm(d_model)
        self.ff = FeedForward(d_model, d_ff)
        self.addnorm2 = AddNorm(d_model)

    def forward(self, x):
        x = self.addnorm1(x, self.attn(x))  # Attention + Residual + LayerNorm
        x = self.addnorm2(x, self.ff(x))    # FeedForward + Residual + LayerNorm
        return x


7. Zusammenbau des NanoTransformers (nur Decoder)

In [8]:

class NanoTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, d_ff, max_len, num_layers):
        super().__init__()
        self.embed = TokenAndPositionalEmbedding(vocab_size, d_model, max_len)
        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, n_heads, d_ff) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.output_proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        logits = self.output_proj(x)
        return logits

8. Trainieren des Modells

In [12]:

import torch.optim as optim
from tqdm import tqdm


def train(model, dataloader, vocab_size, device, epochs=10, lr=1e-4):
    # Wandb initialisieren, nur einmal zu Beginn des Trainings
    wandb.init(project="nano-transformer", config={
        "epochs": epochs,
        "lr": lr,
        "batch_size": dataloader.batch_size,
        "seq_len": dataloader.dataset.seq_len,
        "vocab_size": vocab_size
    })
    
    # Modell auf das richtige Gerät (GPU oder CPU) verschieben
    model = model.to(device)
    
    # Optimizer und Loss-Funktion
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    model.train()  # Modell in Trainingsmodus versetzen
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc="Training Progress"):
            # Eingabe- und Ziel-Tensoren
            inputs = batch[:, :-1].to(device)
            targets = batch[:, 1:].to(device)

            # Vorwärtsdurchlauf
            logits = model(inputs)
            logits = logits.view(-1, vocab_size)  # Umformen für CrossEntropyLoss
            targets = targets.view(-1)  # Umformen für CrossEntropyLoss

            # Verlustberechnung
            loss = loss_fn(logits, targets)

            # Backpropagation und Optimierung
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Gesamten Verlust summieren
            total_loss += loss.item()

        # Durchschnittlichen Verlust für die Epoche berechnen
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")

        # Verlust an Wandb senden
        wandb.log({"epoch": epoch+1, "loss": avg_loss})
    
    # Modell speichern nach Training (optional)
    torch.save(model.state_dict(), "nano_transformer_trained.pth")




9. Dataset Integration

In [ ]:


from datasets import load_dataset



# 1. Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# 2. Tiny Shakespeare Dataset laden
dataset = load_dataset("tiny_shakespeare", split="train", trust_remote_code=True)
text = dataset['text'][0]

# 3. Text in Token IDs umwandeln
tokens = tokenizer.encode(text, truncation=False)
tokens = torch.tensor(tokens)

# 4. Dataset Klasse
class TextDataset(Dataset):
    def __init__(self, tokens, seq_len):
        self.tokens = tokens
        self.seq_len = seq_len

    def __len__(self):
        return len(self.tokens) - self.seq_len

    def __getitem__(self, idx):
        return self.tokens[idx:idx+self.seq_len+1]

# 5. DataLoader bauen
seq_len = 64
batch_size = 32
dataset = TextDataset(tokens, seq_len)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print("DataLoader bereit!")

# 6. Den Modell (Mini NanoTransformer)
vocab_size = tokenizer.vocab_size
d_model = 32
n_heads = 1
d_ff = 256
max_len = 64
num_layers = 1

model = NanoTransformer(vocab_size, d_model, n_heads, d_ff, max_len, num_layers)


Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors


DataLoader bereit!


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, dataloader, vocab_size, device, epochs=10, lr=3e-4)



wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,56
loss,2.64159


Training Progress:  10%|█         | 989/9435 [00:12<01:42, 82.05it/s]


KeyboardInterrupt: 

In [15]:


from datasets import load_dataset



# 1. Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# 2. Tiny Shakespeare Dataset laden
dataset = load_dataset("tiny_shakespeare", split="train", trust_remote_code=True)
text = dataset['text'][0]

# 3. Text in Token IDs umwandeln
tokens = tokenizer.encode(text, truncation=False)
tokens = torch.tensor(tokens)

# 4. Dataset Klasse
class TextDataset(Dataset):
    def __init__(self, tokens, seq_len):
        self.tokens = tokens
        self.seq_len = seq_len

    def __len__(self):
        return len(self.tokens) - self.seq_len

    def __getitem__(self, idx):
        return self.tokens[idx:idx+self.seq_len+1]

# 5. DataLoader bauen
seq_len = 64
batch_size = 32
dataset = TextDataset(tokens, seq_len)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print("DataLoader bereit!")


d_model = 128
n_heads = 4
d_ff = 512
num_layers = 2
seq_len = 128 # 128 Token pro Sequenz
max_len = 128 # 128 Token pro Sequenz


model = NanoTransformer(vocab_size, d_model, n_heads, d_ff, max_len, num_layers)


Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors


DataLoader bereit!


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, dataloader, vocab_size, device, epochs=10, lr=3e-4)



Training Progress:  81%|████████  | 7654/9435 [02:10<00:30, 58.77it/s]


KeyboardInterrupt: 

## MaskedSelfAttention, AddNorm und FeedForward ersetzen durch nn.TransformerDecoderLayer

In [13]:
from tqdm import tqdm
#  NanoTransformer mit PyTorch Decoder
from torch.nn import TransformerDecoder, TransformerDecoderLayer

class NanoTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, d_ff, max_len, num_layers):
        super().__init__()
        self.embed = TokenAndPositionalEmbedding(vocab_size, d_model, max_len)

        decoder_layer = TransformerDecoderLayer(d_model, n_heads, d_ff, batch_first=True)
        self.decoder = TransformerDecoder(decoder_layer, num_layers)

        self.norm = nn.LayerNorm(d_model)
        self.output_proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        T = x.size(1)

        # Causal Mask erstellen
        causal_mask = torch.triu(torch.ones(T, T), diagonal=1).bool().to(x.device)

        # Dummy Memory
        memory = torch.zeros(x.size(0), 1, x.size(2)).to(x.device)

        x = self.decoder(tgt=x, memory=memory, tgt_mask=causal_mask)
        x = self.norm(x)
        logits = self.output_proj(x)
        return logits

# Training und Evaluierung
def evaluate(model, dataloader, vocab_size, device):
    model.eval()  # Wechsel in den Evaluationsmodus
    total_loss = 0
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():  # Kein Gradientenberechnung
        for batch in dataloader:
            inputs = batch[:, :-1].to(device)
            targets = batch[:, 1:].to(device)

            logits = model(inputs)
            logits = logits.view(-1, vocab_size)
            targets = targets.view(-1)

            loss = loss_fn(logits, targets)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

def train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=10, lr=1e-4):
    wandb.init(project="nano-transformer", config={
        "epochs": epochs,
        "lr": lr,
        "batch_size": train_dataloader.batch_size,
        "seq_len": train_dataloader.dataset.seq_len,
        "vocab_size": vocab_size,
        "dataset": "tiny_shakespeare",
        "d_ff": d_ff,
        "max_len": max_len,
        "num_layers": num_layers
    })

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
            inputs = batch[:, :-1].to(device)
            targets = batch[:, 1:].to(device)

            logits = model(inputs)
            logits = logits.view(-1, vocab_size)
            targets = targets.view(-1)

            loss = loss_fn(logits, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch+1}: Training Loss = {avg_train_loss:.4f}")

        # Evaluierung nach jedem Trainingsepochendurchgang
        avg_eval_loss = evaluate(model, eval_dataloader, vocab_size, device)
        print(f"Epoch {epoch+1}: Evaluation Loss = {avg_eval_loss:.4f}")

        # Logs an WandB senden
        wandb.log({
            "epoch": epoch+1,
            "train_loss": avg_train_loss,
            "eval_loss": avg_eval_loss
        })

    # Modell speichern nach Training
    torch.save(model.state_dict(), "nano_transformer_trained.pth")

# Dataset vorbereiten
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("tiny_shakespeare", split="train", trust_remote_code=True)
text = dataset['text'][0]
tokens = tokenizer.encode(text, truncation=False)
tokens = torch.tensor(tokens)

class TextDataset(Dataset):
    def __init__(self, tokens, seq_len):
        self.tokens = tokens
        self.seq_len = seq_len

    def __len__(self):
        return len(self.tokens) - self.seq_len

    def __getitem__(self, idx):
        return self.tokens[idx:idx+self.seq_len+1]

seq_len = 64
batch_size = 32
dataset = TextDataset(tokens, seq_len)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Erstellen eines Evaluationsdatensatzes (z.B. indem wir die ersten 20% des Datensatzes verwenden)
eval_dataset = TextDataset(tokens[dataset.seq_len:], seq_len)  # Hier könnten auch speziellere Evaluierungsdaten genutzt werden
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

print("DataLoader bereit!")

# Modell konfigurieren
vocab_size = tokenizer.vocab_size
d_model = 32
n_heads = 1
d_ff = 256
max_len = 64
num_layers = 1

model = NanoTransformer(vocab_size, d_model, n_heads, d_ff, max_len, num_layers)

# Training starten
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=100, lr=3e-4)


Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


DataLoader bereit!


Epoch 1: 100%|██████████| 9435/9435 [01:58<00:00, 79.61it/s]


Epoch 1: Training Loss = 5.0648
Epoch 1: Evaluation Loss = 4.2630


Epoch 2: 100%|██████████| 9435/9435 [02:05<00:00, 75.08it/s]


...

Epoch 11: 100%|██████████| 9435/9435 [01:54<00:00, 82.76it/s]


Epoch 11: Training Loss = 3.1437
Epoch 11: Evaluation Loss = 3.1232


Epoch 12: 100%|██████████| 9435/9435 [02:00<00:00, 78.04it/s]


Epoch 12: Training Loss = 3.1115


KeyboardInterrupt: 

### lr statt 0.0003 zu 0.003

In [14]:
# Erstellen eines Evaluationsdatensatzes (z.B. indem wir die ersten 20% des Datensatzes verwenden)
eval_dataset = TextDataset(tokens[dataset.seq_len:], seq_len)  # Hier könnten auch speziellere Evaluierungsdaten genutzt werden
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

print("DataLoader bereit!")

# Modell konfigurieren
vocab_size = tokenizer.vocab_size
d_model = 32
n_heads = 1
d_ff = 256
max_len = 64
num_layers = 1

model = NanoTransformer(vocab_size, d_model, n_heads, d_ff, max_len, num_layers)

# Training starten
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=10, lr=3e-3)


DataLoader bereit!


epoch,▁▂▂▃▄▅▅▆▇▇█
eval_loss,█▅▄▃▃▂▂▂▁▁▁
train_loss,█▄▃▃▂▂▂▁▁▁▁
epoch,11
eval_loss,3.12321
train_loss,3.14373


Epoch 1: 100%|██████████| 9435/9435 [02:01<00:00, 77.36it/s]


Epoch 1: Training Loss = 3.8809
Epoch 1: Evaluation Loss = 3.4163


Epoch 2: 100%|██████████| 9435/9435 [01:57<00:00, 80.60it/s]


...

Epoch 8: Training Loss = 2.6617
Epoch 8: Evaluation Loss = 2.6357


Epoch 9: 100%|██████████| 9435/9435 [01:49<00:00, 85.87it/s]


Epoch 9: Training Loss = 2.6311
Epoch 9: Evaluation Loss = 2.6102


Epoch 10: 100%|██████████| 9435/9435 [01:53<00:00, 82.88it/s]


Epoch 10: Training Loss = 2.6050
Epoch 10: Evaluation Loss = 2.5891


### num_layers =von 1 zu  2 / n_heads = von 1 zu 4


In [15]:
# Erstellen eines Evaluationsdatensatzes (z.B. indem wir die ersten 20% des Datensatzes verwenden)
eval_dataset = TextDataset(tokens[dataset.seq_len:], seq_len)  # Hier könnten auch speziellere Evaluierungsdaten genutzt werden
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

print("DataLoader bereit!")

# Modell konfigurieren
vocab_size = tokenizer.vocab_size
d_model = 32
n_heads = 4
d_ff = 256
max_len = 64
num_layers = 2

model = NanoTransformer(vocab_size, d_model, n_heads, d_ff, max_len, num_layers)

# Training starten
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=10, lr=3e-3)


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


DataLoader bereit!


epoch,▁▂▃▃▄▅▆▆▇█
eval_loss,█▅▄▃▂▂▂▁▁▁
train_loss,█▄▃▃▂▂▂▁▁▁
epoch,10
eval_loss,2.58912
train_loss,2.60498


Epoch 1: 100%|██████████| 9435/9435 [02:18<00:00, 68.04it/s]


Epoch 1: Training Loss = 3.8056
Epoch 1: Evaluation Loss = 3.2602


...

Epoch 8: Training Loss = 2.3576
Epoch 8: Evaluation Loss = 2.3354


Epoch 9: 100%|██████████| 9435/9435 [02:12<00:00, 71.22it/s]


Epoch 9: Training Loss = 2.3321
Epoch 9: Evaluation Loss = 2.3148


Epoch 10: 100%|██████████| 9435/9435 [02:21<00:00, 66.45it/s]


Epoch 10: Training Loss = 2.3096
Epoch 10: Evaluation Loss = 2.2918


### max_len = von 64 zu 128 / d_ff = von 256 zu 512  

In [16]:
# Erstellen eines Evaluationsdatensatzes (z.B. indem wir die ersten 20% des Datensatzes verwenden)
eval_dataset = TextDataset(tokens[dataset.seq_len:], seq_len)  # Hier könnten auch speziellere Evaluierungsdaten genutzt werden
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

print("DataLoader bereit!")

# Modell konfigurieren
vocab_size = tokenizer.vocab_size
d_model = 32
n_heads = 4
d_ff = 512  
max_len = 128
num_layers = 2

model = NanoTransformer(vocab_size, d_model, n_heads, d_ff, max_len, num_layers)

# Training starten
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=10, lr=3e-3)


DataLoader bereit!


epoch,▁▂▃▃▄▅▆▆▇█
eval_loss,█▄▃▃▂▂▂▁▁▁
train_loss,█▄▃▂▂▂▁▁▁▁
epoch,10
eval_loss,2.2918
train_loss,2.30964


Epoch 1: 100%|██████████| 9435/9435 [02:24<00:00, 65.20it/s]


Epoch 1: Training Loss = 3.8457
Epoch 1: Evaluation Loss = 3.2873


Epoch 2: 100%|██████████| 9435/9435 [02:20<00:00, 67.34it/s]


...

Epoch 8: Training Loss = 2.2949
Epoch 8: Evaluation Loss = 2.2728


Epoch 9: 100%|██████████| 9435/9435 [02:19<00:00, 67.45it/s]


Epoch 9: Training Loss = 2.2676
Epoch 9: Evaluation Loss = 2.2426


Epoch 10: 100%|██████████| 9435/9435 [02:17<00:00, 68.40it/s]


Epoch 10: Training Loss = 2.2442
Epoch 10: Evaluation Loss = 2.2160


### d_model = von 32 zu 128 / num_layers = von 2 zu 4

In [17]:
# Modell konfigurieren
vocab_size = tokenizer.vocab_size
d_model = 128
n_heads = 4
d_ff = 512  
max_len = 128
num_layers = 4

model = NanoTransformer(vocab_size, d_model, n_heads, d_ff, max_len, num_layers)

# Training starten
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=10, lr=3e-3)


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▂▃▃▄▅▆▆▇█
eval_loss,█▄▃▂▂▂▂▁▁▁
train_loss,█▄▃▂▂▂▁▁▁▁
epoch,10
eval_loss,2.216
train_loss,2.24425


Epoch 1: 100%|██████████| 9435/9435 [03:48<00:00, 41.22it/s]


Epoch 1: Training Loss = 5.9286
Epoch 1: Evaluation Loss = 5.8530


Epoch 2: 100%|██████████| 9435/9435 [03:46<00:00, 41.68it/s]


...

Epoch 8: Training Loss = 4.0530
Epoch 8: Evaluation Loss = 3.9643


Epoch 9: 100%|██████████| 9435/9435 [03:32<00:00, 44.38it/s]


Epoch 9: Training Loss = 3.9459
Epoch 9: Evaluation Loss = 3.8622


Epoch 10: 100%|██████████| 9435/9435 [03:32<00:00, 44.36it/s]


Epoch 10: Training Loss = 3.8531
Epoch 10: Evaluation Loss = 3.8033


### Optimierungsalgorithmus von  Adam zu SGD

In [19]:
from tqdm import tqdm
#  NanoTransformer mit PyTorch Decoder
from torch.nn import TransformerDecoder, TransformerDecoderLayer

class NanoTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, d_ff, max_len, num_layers):
        super().__init__()
        self.embed = TokenAndPositionalEmbedding(vocab_size, d_model, max_len)

        decoder_layer = TransformerDecoderLayer(d_model, n_heads, d_ff, batch_first=True)
        self.decoder = TransformerDecoder(decoder_layer, num_layers)

        self.norm = nn.LayerNorm(d_model)
        self.output_proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        T = x.size(1)

        # Causal Mask erstellen
        causal_mask = torch.triu(torch.ones(T, T), diagonal=1).bool().to(x.device)

        # Dummy Memory
        memory = torch.zeros(x.size(0), 1, x.size(2)).to(x.device)

        x = self.decoder(tgt=x, memory=memory, tgt_mask=causal_mask)
        x = self.norm(x)
        logits = self.output_proj(x)
        return logits

# Training und Evaluierung
def evaluate(model, dataloader, vocab_size, device):
    model.eval()  # Wechsel in den Evaluationsmodus
    total_loss = 0
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():  # Kein Gradientenberechnung
        for batch in dataloader:
            inputs = batch[:, :-1].to(device)
            targets = batch[:, 1:].to(device)

            logits = model(inputs)
            logits = logits.view(-1, vocab_size)
            targets = targets.view(-1)

            loss = loss_fn(logits, targets)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

def train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=10, lr=1e-4):
    wandb.init(project="nano-transformer", config={
        "epochs": epochs,
        "lr": lr,
        "batch_size": train_dataloader.batch_size,
        "seq_len": train_dataloader.dataset.seq_len,
        "vocab_size": vocab_size,
        "dataset": "tiny_shakespeare",
        "d_ff": d_ff,
        "max_len": max_len,
        "num_layers": num_layers
    })

    model = model.to(device)
    optimizer =  optim.SGD(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
            inputs = batch[:, :-1].to(device)
            targets = batch[:, 1:].to(device)

            logits = model(inputs)
            logits = logits.view(-1, vocab_size)
            targets = targets.view(-1)

            loss = loss_fn(logits, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch+1}: Training Loss = {avg_train_loss:.4f}")

        # Evaluierung nach jedem Trainingsepochendurchgang
        avg_eval_loss = evaluate(model, eval_dataloader, vocab_size, device)
        print(f"Epoch {epoch+1}: Evaluation Loss = {avg_eval_loss:.4f}")

        # Logs an WandB senden
        wandb.log({
            "epoch": epoch+1,
            "train_loss": avg_train_loss,
            "eval_loss": avg_eval_loss
        })

    # Modell speichern nach Training
    torch.save(model.state_dict(), "nano_transformer_trained.pth")

# Dataset vorbereiten
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("tiny_shakespeare", split="train", trust_remote_code=True)
text = dataset['text'][0]
tokens = tokenizer.encode(text, truncation=False)
tokens = torch.tensor(tokens)

class TextDataset(Dataset):
    def __init__(self, tokens, seq_len):
        self.tokens = tokens
        self.seq_len = seq_len

    def __len__(self):
        return len(self.tokens) - self.seq_len

    def __getitem__(self, idx):
        return self.tokens[idx:idx+self.seq_len+1]

seq_len = 64
batch_size = 32
dataset = TextDataset(tokens, seq_len)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Erstellen eines Evaluationsdatensatzes (z.B. indem wir die ersten 20% des Datensatzes verwenden)
eval_dataset = TextDataset(tokens[dataset.seq_len:], seq_len)  # Hier könnten auch speziellere Evaluierungsdaten genutzt werden
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

print("DataLoader bereit!")

# Modell konfigurieren
vocab_size = tokenizer.vocab_size
d_model = 32
n_heads = 4
d_ff = 512  
max_len = 128
num_layers = 2

model = NanoTransformer(vocab_size, d_model, n_heads, d_ff, max_len, num_layers)

# Training starten
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=10, lr=3e-3)


Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


DataLoader bereit!


epoch,▁▂▃▃▄▅▆▆▇█
eval_loss,█▆▅▄▃▃▂▂▁▁
train_loss,█▅▄▃▃▂▂▁▁▁
epoch,10
eval_loss,7.46348
train_loss,7.50356


Epoch 1: 100%|██████████| 9435/9435 [02:15<00:00, 69.82it/s]


Epoch 1: Training Loss = 8.3323
Epoch 1: Evaluation Loss = 7.5116


...

Epoch 8: Training Loss = 6.1660
Epoch 8: Evaluation Loss = 6.1289


Epoch 9: 100%|██████████| 9435/9435 [02:14<00:00, 69.92it/s]


Epoch 9: Training Loss = 6.0917
Epoch 9: Evaluation Loss = 6.0546


Epoch 10: 100%|██████████| 9435/9435 [02:14<00:00, 70.32it/s]


Epoch 10: Training Loss = 6.0164
Epoch 10: Evaluation Loss = 5.9775


### batch_size = von 32 zu  64


In [21]:
from tqdm import tqdm
#  NanoTransformer mit PyTorch Decoder
from torch.nn import TransformerDecoder, TransformerDecoderLayer

class NanoTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, d_ff, max_len, num_layers):
        super().__init__()
        self.embed = TokenAndPositionalEmbedding(vocab_size, d_model, max_len)

        decoder_layer = TransformerDecoderLayer(d_model, n_heads, d_ff, batch_first=True)
        self.decoder = TransformerDecoder(decoder_layer, num_layers)

        self.norm = nn.LayerNorm(d_model)
        self.output_proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        T = x.size(1)

        # Causal Mask erstellen
        causal_mask = torch.triu(torch.ones(T, T), diagonal=1).bool().to(x.device)

        # Dummy Memory
        memory = torch.zeros(x.size(0), 1, x.size(2)).to(x.device)

        x = self.decoder(tgt=x, memory=memory, tgt_mask=causal_mask)
        x = self.norm(x)
        logits = self.output_proj(x)
        return logits

# Training und Evaluierung
def evaluate(model, dataloader, vocab_size, device):
    model.eval()  # Wechsel in den Evaluationsmodus
    total_loss = 0
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():  # Kein Gradientenberechnung
        for batch in dataloader:
            inputs = batch[:, :-1].to(device)
            targets = batch[:, 1:].to(device)

            logits = model(inputs)
            logits = logits.view(-1, vocab_size)
            targets = targets.view(-1)

            loss = loss_fn(logits, targets)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

def train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=10, lr=1e-4):
    wandb.init(project="nano-transformer", config={
        "epochs": epochs,
        "lr": lr,
        "batch_size": train_dataloader.batch_size,
        "seq_len": train_dataloader.dataset.seq_len,
        "vocab_size": vocab_size,
        "dataset": "tiny_shakespeare",
        "d_ff": d_ff,
        "max_len": max_len,
        "num_layers": num_layers
    })

    model = model.to(device)
    optimizer =  optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
            inputs = batch[:, :-1].to(device)
            targets = batch[:, 1:].to(device)

            logits = model(inputs)
            logits = logits.view(-1, vocab_size)
            targets = targets.view(-1)

            loss = loss_fn(logits, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch+1}: Training Loss = {avg_train_loss:.4f}")

        # Evaluierung nach jedem Trainingsepochendurchgang
        avg_eval_loss = evaluate(model, eval_dataloader, vocab_size, device)
        print(f"Epoch {epoch+1}: Evaluation Loss = {avg_eval_loss:.4f}")

        # Logs an WandB senden
        wandb.log({
            "epoch": epoch+1,
            "train_loss": avg_train_loss,
            "eval_loss": avg_eval_loss
        })

    # Modell speichern nach Training
    torch.save(model.state_dict(), "nano_transformer_trained.pth")

# Dataset vorbereiten
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("tiny_shakespeare", split="train", trust_remote_code=True)
text = dataset['text'][0]
tokens = tokenizer.encode(text, truncation=False)
tokens = torch.tensor(tokens)

class TextDataset(Dataset):
    def __init__(self, tokens, seq_len):
        self.tokens = tokens
        self.seq_len = seq_len

    def __len__(self):
        return len(self.tokens) - self.seq_len

    def __getitem__(self, idx):
        return self.tokens[idx:idx+self.seq_len+1]

seq_len = 64
batch_size = 64
dataset = TextDataset(tokens, seq_len)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Erstellen eines Evaluationsdatensatzes (z.B. indem wir die ersten 20% des Datensatzes verwenden)
eval_dataset = TextDataset(tokens[dataset.seq_len:], seq_len)  # Hier könnten auch speziellere Evaluierungsdaten genutzt werden
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

print("DataLoader bereit!")

# Modell konfigurieren
vocab_size = tokenizer.vocab_size
d_model = 32
n_heads = 4
d_ff = 512  
max_len = 128
num_layers = 2

model = NanoTransformer(vocab_size, d_model, n_heads, d_ff, max_len, num_layers)

# Training starten
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, train_dataloader, eval_dataloader, vocab_size, device, epochs=10, lr=3e-3)


Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


DataLoader bereit!


Epoch 1: 100%|██████████| 4718/4718 [02:14<00:00, 35.20it/s]


Epoch 1: Training Loss = 3.9587
Epoch 1: Evaluation Loss = 3.3660


Epoch 2: 100%|██████████| 4718/4718 [02:35<00:00, 30.32it/s]


...

Epoch 8: Training Loss = 2.3252
Epoch 8: Evaluation Loss = 2.3018


Epoch 9: 100%|██████████| 4718/4718 [02:40<00:00, 29.37it/s]


Epoch 9: Training Loss = 2.2955
Epoch 9: Evaluation Loss = 2.2779


Epoch 10: 100%|██████████| 4718/4718 [02:36<00:00, 30.13it/s]


Epoch 10: Training Loss = 2.2699
Epoch 10: Evaluation Loss = 2.2472
